In [2]:
import time
from api.interface import *
from datetime import datetime

Let's start with Kafka

To run server & consumer, open three terminals in order (in kafka directory)
- bin/zookeeper-server-start.sh config/zookeeper.properties 
- bin/kafka-server-start.sh config/server.properties
- bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testing

In [2]:
# Start everything up as before
config = Config(
    provider = 'kafka', server = 'localhost:9092'
)
consumer = Consumer(config)
consumer.add_topic("fashion_mnist")

Run this to send a stream of images and indices to fashion_mnist:

```python pt3/simulated_client.py```

Now, we can start streaming messages into a function that classifies the image.

For now, we'll do it in the main thread so we can see the results right here

But this is blocking. In the cell after this one I'll trigger this in the OOP way so that it starts a thread automatically

In [3]:
import numpy as np
import pickle, codecs, json, torch

from cnn.model import generate_model_by_name
from cnn.api import predict_numpy_images
from api.utils import generate_consumer, direct_messages


model = generate_model_by_name(num_classes=10, input_size=[1, 28, 28], name="default")
# Load weights here

def classification_callback_function(message):
    index, image = json.loads(message.value.decode())
    image = np.array(image)
    output = predict_numpy_images([image], model)
    pred = torch.argmax(output)
    print(f"Image with id {index} classified as: {pred}")

    
con = generate_consumer(config, "fashion_mnist")
direct_messages(config, con, classification_callback_function)

# In fact starting this in a blocking way gives this error...

Image with id 6997 classified as: 7


AttributeError: '_MainThread' object has no attribute 'stopped'

Instead, what we should do is redirect our predictions to another kafka topic

Consumer.start_stream() does what we did above in an async way

Start a listening terminal for the ```fashion_predictions``` topic to see the results coming in:

```bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic fashion_predictions```

In [3]:
import threading

config = Config(
    provider = 'kafka', server = 'localhost:9092'
)

consumer = Consumer(config)
consumer.add_topic("fashion_mnist")

results_producer = Producer(config)
results_producer.add_topic('fashion_predictions')

def classify_and_report_callback_function(msg):
    index, image = json.loads(msg.value.decode())
    img = np.array(image)
    output = predict_numpy_images([img], model)
    pred = torch.argmax(output).item()

    results_producer.send_message(
        message=(index, pred),
        topic_name="fashion_predictions"
    )

# consumer.kill_stream("fashion_mnist", "classification")

consumer.start_stream(
    topic_name="fashion_mnist", 
    callback_function=classify_and_report_callback_function, 
    stream_name="classification"
)

Started streaming messages from fashion_mnist to <function classify_and_report_callback_function at 0x7f85187a41f0>


In [26]:
x, y, z, n, = 16, 16, 16, 5

import numpy as np
a = np.array([[[(i, j, k) for i in range(x+1) if (x+y+z)!=n ] for j in range(y+1)] for k in range(z+1)])
a.reshape(-1, 3)

array([[ 0,  0,  0],
       [ 1,  0,  0],
       [ 2,  0,  0],
       ...,
       [14, 16, 16],
       [15, 16, 16],
       [16, 16, 16]])

In [10]:
[i+1 for i in range(x) if i !=7]

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]

In [3]:



    [
        [
            [
                (i,j,k) for i in range(x+1)
            ] for j in range(y+1)
        ] for k in range(z+1)
    ] if (i+j+k)!=n
]

SyntaxError: invalid syntax (<ipython-input-3-c6bcb9be94b5>, line 11)